# Correlated data

A simplified approach to generate correlated data - that is in line with the simulated zero curves.  
Check the correlation of mortgage interest rates and swap rates

Several methods and options exist:
- Box-Muller Method
- Iman-Conover Method
- Vasicek factor 2 model
- Georgian Brown Movements

The most simple option that still takes care of the correlation between variables is to use historical monte carlo.


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import numpy as np

# Find the root directory by traversing up the directory tree
def find_project_root(current_path):
    if os.path.exists(os.path.join(current_path, "README.md")):
        return current_path
    parent_path = os.path.dirname(current_path)
    if parent_path == current_path:
        raise ValueError("Project root not found.")
    return find_project_root(parent_path)


current_dir = os.getcwd()
project_root = find_project_root(current_dir)
sys.path.append(str(project_root))
print(f"Project root: {project_root}")

from src.data.zerocurve import Zerocurve
from src.data.interest import Interest

Project root: f:\OneDrive\Documents\GitHub\rl-alm


f:\OneDrive\Documents\GitHub\rl-alm\rl-alm\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.6) or chardet (None)/charset_normalizer (3.3.0) doesn't match a supported version!
  warnings.warn(


In [3]:
zerocurve = Zerocurve()
zerocurve.read_data()


2023-10-07 11:32:53.340 | INFO     | src.data.dataset:read_data:45 - reading zerocurve data from ESW.


<Response [200]>

In [4]:
from pandas.tseries.offsets import BDay
yield_data = zerocurve.yield_data
# yield_data_change = yield_data.pct_change().dropna()
yield_data_change = (yield_data - yield_data.shift()).dropna()
chg = yield_data_change.sample(1, replace = True, random_state = 1)
r1 = yield_data.iloc[-1] + yield_data_change.sample(
            1, replace=True, random_state=1
        )
print(r1.values)
last_day = yield_data.index[-1]
print(last_day)
next_day = last_day + BDay(1)
yield_data.loc[next_day] = r1.values[0]
yield_data
#print(yield_data.tail(1))
#print(yield_data.iloc[-1] + yield_data.iloc[-1] * chg)

#growth_vanilla = df["Growth Rate - Vanilla"].dropna(
#    ).sample(1000, replace=True, random_state=1)

[[3.70546434 3.70546434 3.57037044 3.36198869 3.14379509 2.94721953
  2.785259   2.57147367 2.46849133 2.56727582 2.71475695 2.97579006
  3.19984106 3.25239269 2.59354025]]
2023-09-29 00:00:00


tenor,0,3,6,9,12,15,18,24,36,48,60,84,120,180,360
rate_dt,,,,,,,,,,,,,,,
2010-06-01,0.198986,0.198986,0.233508,0.327186,0.466401,0.639714,0.837570,1.276560,2.182619,2.972319,3.576273,4.253072,4.429599,4.094338,3.695506
2010-06-02,0.195494,0.195494,0.240696,0.340034,0.481365,0.654421,0.850553,1.284313,2.183005,2.976086,3.592475,4.301162,4.495208,4.101261,3.559698
2010-06-03,0.197093,0.197093,0.255942,0.366852,0.518021,0.699466,0.902781,1.348057,2.262184,3.064966,3.687989,4.404611,4.601564,4.199481,3.637807
2010-06-04,0.194955,0.194955,0.245217,0.348856,0.493929,0.670320,0.869504,1.308855,2.217980,3.021255,3.647283,4.370987,4.572712,4.164582,3.584203
2010-06-07,0.190019,0.190019,0.245832,0.354690,0.504743,0.685941,0.889797,1.338168,2.263965,3.081217,3.717309,4.446700,4.623877,4.135585,3.424269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-26,3.737301,3.737301,3.656557,3.444470,3.200715,2.974377,2.786112,2.537741,2.417661,2.521918,2.675016,2.939314,3.159040,3.195138,2.481582
2023-09-27,3.730413,3.730413,3.636575,3.421566,3.179101,2.955331,2.769598,2.524639,2.406130,2.510145,2.663490,2.929774,3.153697,3.196498,2.490209
2023-09-28,3.721660,3.721660,3.638389,3.462015,3.262490,3.075940,2.918661,2.706421,2.600821,2.701685,2.854605,3.126773,3.358782,3.404238,2.674394


In [5]:
df_z10 = df_z[df_z['tenor']=='10 Years']
df_z10 = df_z10.drop(['value_dt','tenor'], axis = 'columns')
df_z10 = df_z10.resample('M').mean().reset_index()
df_z10['rate_dt'] = df_z10['rate_dt'].to_numpy().astype('datetime64[M]')
df_z10 = df_z10.set_index('rate_dt')
df_z10 = df_z10.rename(columns = {'rate':'interest'})
df_z10['type'] = 'Swap'
df_z10 = df_z10.loc[df_i.index.min(): df_i.index.max()]
df_report = pd.concat([df_i, df_z10])
sns.lineplot(data = df_report, x = df_report.index, y = 'interest', hue = 'type');

NameError: name 'df_z' is not defined

In [6]:
# Define the number of time periods and factors/instruments
num_periods = 10
num_factors = 3

# Simulate the factor values using a multivariate normal distribution
mean = np.zeros(num_factors)
covariance = np.array([[1.0, 0.5, 0.3], [0.5, 1.0, 0.2], [0.3, 0.2, 1.0]])
factors = np.random.multivariate_normal(mean, covariance, num_periods)

# Simulate the zero curves based on the simulated factor values
zero_curves = np.zeros((num_periods, num_factors))
for i in range(num_periods):
    zero_curves[i,:] = np.exp(-0.05 * (i+1) * factors[i,:])

# Simulate the data based on the simulated zero curves and assumed correlations
correlation_matrix = np.array([[1.0, 0.8, 0.5], [0.8, 1.0, 0.4], [0.5, 0.4, 1.0]])
simulated_data = np.zeros((num_periods, num_factors))
for i in range(num_periods):
    simulated_data[i,:] = norm.ppf(np.random.rand(num_factors))
    simulated_data[i,:] = np.linalg.cholesky(correlation_matrix) @ simulated_data[i,:]
    simulated_data[i,:] = np.exp(simulated_data[i,:] * np.sqrt(1.0 - np.diag(correlation_matrix))) * zero_curves[i,:]

print(simulated_data)   

NameError: name 'norm' is not defined